## BFS

### Iterativo

In [3]:
graph = {'A': set(['B', 'C']),
         'B': set(['A', 'D', 'E']),
         'C': set(['A', 'F']),
         'D': set(['B']),
         'E': set(['B', 'F']),
         'F': set(['C', 'E'])}

def dfs(graph, start):
    visited, stack = set(), [start]
    while stack:
        vertex = stack.pop()
        if vertex not in visited:
            visited.add(vertex)
            stack.extend(graph[vertex] - visited)
    return visited

print(dfs(graph, 'C'))

{'F', 'B', 'D', 'C', 'E', 'A'}


### Paralelo - MapReduce

In [25]:
from pyspark import SparkFiles
from operator import add

sc = SparkContext.getOrCreate()

vertex = "C"
vertexRDD = sc.parallelize(['C'], 4)   

graph = [{'A': set(['B', 'C', 'E']),
         'B': set(['A', 'D', 'E']),
         'C': set(['A', 'F']),
         'D': set(['B']),
         'E': set(['B', 'F']),
         'F': set(['C', 'E'])}]


graphRDD = sc.parallelize(graph, 4)

def visita(graphRDD, nodes):
    q = graphRDD.flatMap(lambda x: x.items())\
              .filter(lambda x: x[0] in nodes)\
              .map(lambda x: x[1])\
              .flatMap(lambda x: x)\
              .map(lambda x: (x,1))\
              .reduceByKey(add)\
              .map(lambda x: x[0])
    return q

def contains(lista, vertice):
    values = lista.filter(lambda x: x == vertice).collect()
    return values

w = [] #contein
lista = ["A"] #relacao visitados
v = "A"
len_lista =100
lista_visitados = [] #todos os caminhos percorridos
while w == [] and len_lista>0:
    lista = visita(graphRDD, lista)
    w = contains(lista, v)
    lista = lista.collect()
    lista_visitados.append(lista)
    if len(lista_visitados) > 3:
        if lista_visitados[-1] == lista_visitados[-2] and lista_visitados[-2] == lista_visitados[-3]:
            break
    print(w)
    print(lista)    #len_lista = lista.count()

[]
['B', 'C', 'E']
['A']
['A', 'F', 'B', 'D', 'E']
